In [4]:
import numpy as np
import pandas as pd
import requests
from scipy import stats
import math

stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

stock_name = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{stock_name}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

peRatio = data["peRatio"]

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']


finalDataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_url_call_stats = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call_stats).json()
    for name in symbol_string.split(','):
        finalDataframe = finalDataframe.append(
        pd.Series(
        [
            name,
            data[name]['quote']['latestPrice'],
            data[name]['quote']['peRatio'],
            'N/A'
            
        ],
        index = my_columns
        ),
        ignore_index = True
        )
        
        

finalDataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
finalDataframe = finalDataframe[finalDataframe['Price-to-Earnings Ratio'] > 0]
finalDataframe = finalDataframe[:50]


def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        
        


valuePerStock = float(portfolio_size) / len(finalDataframe.index)
for i in finalDataframe.index:
    finalDataframe.loc[i , 'Number of Shares to Buy'] = math.floor(valuePerStock / (finalDataframe.loc[i, 'Price']))
    
#symbol = 'AAPL'

#peRatio = data[symbol]['quote']['peRatio']

#pbRatio = data[symbol]['advanced-stats']['priceToBook']

#psRatio = data[symbol]['advanced-stats']['priceToSales']

#evtoebitda = data[symbol]['advanced-stats']['enterpriseValue'] / data[symbol]['advanced-stats']['EBITDA'] 

#evtogrossprofit = data[symbol]['advanced-stats']['enterpriseValue'] / data[symbol]['advanced-stats']['grossProfit']


hqv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

hqvDataframe = pd.DataFrame(columns = hqv_columns)

for symbol_string in symbol_strings:
    batch_api_url_call_stats = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call_stats).json()
    for name in symbol_string.split(','):
        ev = data[name]['advanced-stats']['enterpriseValue']
        ebitda = data[name]['advanced-stats']['EBITDA']
        grossProfit = data[name]['advanced-stats']['grossProfit']
        
        try:
            evtoebitda = ev/ebitda
        except TypeError:
            evtoebitda = np.NaN
            
        try:
            evtogp = ev/grossProfit
        except TypeError:
            evtogp = np.NaN
            
        hqvDataframe = hqvDataframe.append(
        pd.Series(
        [
            name,
            data[name]['quote']['latestPrice'],
            'N/A',
            data[name]['quote']['peRatio'],
            'N/A',
            data[name]['advanced-stats']['priceToBook'],
            'N/A',
            evtoebitda,
            'N/A',
            evtogp,
            'N/A',
            'N/A'
        ],
        index = hqv_columns
        ),
        ignore_index = True
        )
        
        
        
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','EV/EBITDA', 'EV/GP']:
    hqvDataframe[column].fillna(hqvDataframe[column].mean(), inplace = True)
    
hqvDataframe[hqvDataframe.isnull().any(axis=1)]

metrics = {'Price-to-Earnings Ratio' : 'PE Percentile', 
           'Price-to-Book Ratio':'PB Percentile',
           'EV/EBITDA':'EV/EBITDA Percentile', 
           'EV/GP':'EV/GP Percentile'
          }

for metric in metrics.keys():
    for row in hqvDataframe.index:
        hqvDataframe.loc[row, metrics[metric]] = stats.percentileofscore(hqvDataframe[metric], hqvDataframe.loc[row, metric])
        
from statistics import mean

for row in hqvDataframe.index:
    valuePercentiles = []
    for metric in metrics.keys():
        valuePercentiles.append(hqvDataframe.loc[row, metrics[metric]])
    hqvDataframe.loc[row, 'RV Score'] = mean(valuePercentiles)
    
hqvDataframe.sort_values('RV Score', ascending=True, inplace=True)
hqvDataframe = hqvDataframe[:50]
hqvDataframe.reset_index(inplace = True, drop=True)

portfolio_input()

valuePerStock = float(portfolio_size) / len(hqvDataframe.index)
for i in hqvDataframe.index:
    hqvDataframe.loc[i , 'Number of Shares to Buy'] = math.floor(valuePerStock / (hqvDataframe.loc[i, 'Price']))
    
print(hqvDataframe)

Enter the value of your portfolio:1000000
   Ticker   Price Number of Shares to Buy  Price-to-Earnings Ratio  \
0      BA  220.98                      90                   -10.20   
1     LYV   87.71                     228                   -12.16   
2     AIG   43.10                     464                    -7.50   
3     UNM   25.39                     787                     5.50   
4     AAL   17.83                    1121                    -1.25   
5     MET   55.75                     358                     9.47   
6     MCK  182.40                     109                    -7.30   
7     HIG   51.48                     388                    10.41   
8     ALL  106.08                     188                     7.55   
9     AFL   46.97                     425                     7.75   
10    FTI   10.40                    1923                    -0.84   
11      F   11.59                    1725                   -35.78   
12     BK   42.46                     471       